# ツール使用と拡張思考の統合

このノートブックでは、Claude 3.7 Sonnet の拡張思考機能とツール使用機能を効果的に組み合わせる方法について説明します。このレッスンの終わりまでに、次のことを理解できるようになります。

1. 拡張思考とツール使用を連携させる方法
2. シーケンシャルな推論からツール呼び出しのパターンを実装する方法
3. 推論とツールを組み合わせたマルチステップ ワークフローを構築する手法
4. エラーを処理し、問題から回復するための戦略

これらの概念を説明するために、計算機、天気 API、Wikipedia 検索などの基本的なツールを使用したシンプルで実用的な例を使用します。

## 前提条件

- Claude 3.7 の拡張思考機能に関する基本的な理解 (以前のレッスンで説明)
- Bedrock API とユーティリティ関数に関する知識
- Python プログラミングの経験

まず、環境の設定から始めましょう。

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML
import requests
import math
import re
from datetime import datetime

# Import our utility functions from previous lessons
import claude_utils

# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Claude 3.7 Sonnet model ID
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)

## 拡張思考によるツールの使用の理解

Claude 3.7 Sonnet は、拡張思考とツールの使用という 2 つの強力な機能を組み合わせています。これらを効果的に組み合わせると、人間が複雑なタスクに取り組む方法を模倣したワークフローが作成されます。

1. **拡張思考**: クロードは専用の「スクラッチパッド」スペースで問題を段階的に考えます

2. **ツールの使用**: クロードは外部ツールを要求して使用し、情報を収集したりアクションを実行したりします

### シーケンシャル パターン

拡張思考とツールの使用を組み合わせると、クロードは特定のシーケンスに従います。

1. アシスタント ターンごとに 1 つの拡張思考ブロック (スクラッチパッド)

2. アシスタントの 1 つのテキスト応答

3. 可能なツール呼び出しが続きます

つまり、次のようになります。
- 拡張思考とツール呼び出しが交互に表示されることはありません
- 拡張思考が最初に発生し、その後にツール呼び出しが続きます
- 拡張思考自体内でのツール使用はサポートされていません

プロセスの視覚化を以下に示します。

[ユーザー クエリ] → [拡張思考] → [アシスタント応答] → [ツール呼び出し] → [ツール結果] → [次のターン]

これを人間のように考えます1. まず問題を慎重に考える
2. 次に理由を説明する
3. 最後にその理由に基づいて行動する

このパターンは、慎重な推論と外部の情報や機能へのアクセスの両方を必要とするタスクに特に効果的です。

## ツール使用ユーティリティ

このノートブックは、`claude_utils.py` モジュールのいくつかのユーティリティ関数を使用して、Claude のツール使用機能の操作を効率化します。

- `define_tool`: 名前、説明、およびパラメータ スキーマを含むツール仕様を作成します
- `handle_tool_call`: ツール呼び出しを処理し、適切なハンドラー関数にルーティングします
- `calculate`、`get_weather`、`search_wikipedia`: 個々のツール実装
- `get_common_tools`: 事前構成された一般的なツールの辞書を返します

これらのユーティリティ関数を使用すると、実装の詳細に迷うことなく、拡張された思考とツール使用を組み合わせるというコア コンセプトに集中できます。ユース ケースに固有の新しいツールとハンドラーを追加することで、これらのユーティリティ関数を独自のアプリケーションに適応させることができます。

簡単にするために、サンプル ツールでは外部 API を呼び出すのではなくシミュレートされたデータを使用しますが、実際のデータ ソースに接続する場合もパターンは同じです。

それでは、これらのツールを使用して、クロード氏の拡張思考能力がどのように機能するかを見てみましょう。

In [ ]:
# Import our utility functions
from claude_utils import define_tool, handle_tool_call, get_common_tools

# Get the common tools
tools = get_common_tools()
calculator_tool = tools["calculator"]
weather_tool = tools["weather"]
wikipedia_tool = tools["wikipedia"]

# Display the tools
print(f"Loaded {len(tools)} tools:")
for name, tool in tools.items():
    print(f"- {tool['toolSpec']['name']}: {tool['toolSpec']['description']}")

#### ツール ハンドラーを実装しましょう。

In [ ]:
def handle_tool_call(tool_name, tool_input):
    """
    Handle tool calls by executing the appropriate function
    
    Args:
        tool_name (str): Name of the tool to call
        tool_input (dict): Parameters for the tool
        
    Returns:
        str or dict: Result of the tool execution
    """
    try:
        if tool_name == "calculator":
            return calculate(tool_input["expression"])
        elif tool_name == "get_weather":
            return get_weather(tool_input["location"], tool_input.get("unit", "celsius"))
        elif tool_name == "search_wikipedia":
            return search_wikipedia(tool_input["query"], tool_input.get("max_results", 3))
        else:
            return {"error": f"Unknown tool: {tool_name}"}
    except Exception as e:
        return {"error": f"Error executing {tool_name}: {str(e)}"}

def calculate(expression):
    """Simple calculator function"""
    # Remove any potentially unsafe operations
    if any(unsafe in expression for unsafe in ["import", "exec", "eval", "compile", "open", "__"]):
        return {"error": "Unsafe expression"}
    
    try:
        # Use a safer approach to evaluate mathematical expressions
        # This is a simplified version - in production you'd want more safeguards
        allowed_symbols = {
            'sqrt': math.sqrt, 'pi': math.pi, 'e': math.e,
            'sin': math.sin, 'cos': math.cos, 'tan': math.tan,
            'log': math.log, 'log10': math.log10, 'exp': math.exp,
            'floor': math.floor, 'ceil': math.ceil, 'abs': abs
        }
        
        # Replace common math operations with Python syntax
        expression = expression.replace('^', '**')
        result = eval(expression, {"__builtins__": {}}, allowed_symbols)
        return {"result": result}
    except Exception as e:
        return {"error": f"Failed to calculate: {str(e)}"}

def get_weather(location, unit="celsius"):
    """Simulate getting weather data for a location"""
    # In a real application, you would call a weather API
    # For this example, we'll just return simulated data
    
    weather_data = {
        "New York, USA": {"temp": 22, "conditions": "Partly Cloudy", "humidity": 65},
        "London, UK": {"temp": 18, "conditions": "Rainy", "humidity": 78},
        "Tokyo, Japan": {"temp": 26, "conditions": "Sunny", "humidity": 60},
        "Sydney, Australia": {"temp": 24, "conditions": "Clear", "humidity": 55},
        "Paris, France": {"temp": 20, "conditions": "Cloudy", "humidity": 70}
    }
    
    # Default to a generic response if location not found
    data = weather_data.get(location, {"temp": 21, "conditions": "Unknown", "humidity": 65})
    
    # Convert temperature if needed
    if unit.lower() == "fahrenheit":
        data["temp"] = (data["temp"] * 9/5) + 32
    
    return {
        "location": location,
        "temperature": data["temp"],
        "unit": unit,
        "conditions": data["conditions"],
        "humidity": data["humidity"],
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

def search_wikipedia(query, max_results=3):
    """Simulate searching Wikipedia"""
    # In a real application, you would use the Wikipedia API
    # For this example, we'll just return simulated data
    
    wiki_results = {
        "python": [
            {"title": "Python (programming language)", "snippet": "Python is a high-level, interpreted programming language known for its readability and versatility..."},
            {"title": "Python (genus)", "snippet": "Python is a genus of constricting snakes in the Pythonidae family native to the tropics and subtropics..."},
            {"title": "Monty Python", "snippet": "Monty Python was a British surreal comedy troupe formed in 1969..."}
        ],
        "climate change": [
            {"title": "Climate change", "snippet": "Climate change refers to long-term shifts in temperatures and weather patterns..."},
            {"title": "Global warming", "snippet": "Global warming is the long-term heating of Earth's climate system observed since the pre-industrial period..."},
            {"title": "Climate change mitigation", "snippet": "Climate change mitigation consists of actions to limit global warming..."}
        ],
        "artificial intelligence": [
            {"title": "Artificial intelligence", "snippet": "Artificial intelligence (AI) is intelligence demonstrated by machines..."},
            {"title": "Machine learning", "snippet": "Machine learning is a field of inquiry devoted to understanding and building methods that 'learn'..."},
            {"title": "History of artificial intelligence", "snippet": "The history of artificial intelligence began in antiquity, with myths, stories and rumors..."}
        ]
    }
    
    # Handle case-insensitivity with fallbacks
    query_lower = query.lower()
    for key in wiki_results.keys():
        if query_lower in key or key in query_lower:
            query_lower = key
            break
    
    results = wiki_results.get(query_lower, [
        {"title": f"No exact match for '{query}'", "snippet": "Try another search term..."}
    ])
    
    return {
        "query": query,
        "results": results[:max_results],
        "total_results": len(results),
        "search_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

#### ここで、拡張された思考とツールの使用で Claude を呼び出す関数を実装しましょう。

In [ ]:
def invoke_claude_with_tools(prompt, tools, reasoning_budget=4096, max_tokens=1000):
    """
    Invoke Claude with both extended thinking and tool use capabilities
    
    Args:
        prompt (str): The user prompt
        tools (list): List of tools to make available
        reasoning_budget (int): Token budget for reasoning
        max_tokens (int): Maximum tokens for the response
        
    Returns:
        dict: The full API response
    """
    # Create a more descriptive system prompt that explains the available tools
    tool_descriptions = "\n".join([
        f"- {tool['toolSpec']['name']}: {tool['toolSpec']['description']}" 
        for tool in tools
    ])
    
    system_prompt = [{
        "text": f"""You're a helpful AI assistant with the ability to use tools.
        
You have access to the following tools:
{tool_descriptions}

When a user's request requires using one of these tools:
1. First think through what information you need and which tool would be appropriate
2. Then provide a clear explanation to the user about your approach
3. Finally use the appropriate tool by including the necessary parameters

Important: If a question requires calculation, getting weather data, or searching for information,
always use the appropriate tool rather than trying to answer from your knowledge."""
    }]
    
    # Create messages
    messages = [
        {
            "role": "user",
            "content": [{"text": prompt}]
        }
    ]
    
    # Base request parameters
    request_params = {
        "modelId": CLAUDE_37_SONNET_MODEL_ID,
        "messages": messages,
        "system": system_prompt,
        "inferenceConfig": {
            "temperature": 1.0,  # Must be 1.0 when reasoning is enabled
            "maxTokens": max(reasoning_budget + 1, max_tokens)
        },
        "additionalModelRequestFields": {
            "reasoning_config": {
                "type": "enabled",
                "budget_tokens": reasoning_budget
            }
        },
        "toolConfig": {
            "tools": tools
        }
    }
    
    # Invoke the model
    start_time = time.time()
    response = bedrock_runtime.converse(**request_params)
    elapsed_time = time.time() - start_time
    
    # Add elapsed time to response for reference
    response["_elapsed_time"] = elapsed_time
    
    # Debug: print the raw response structure to help diagnose parsing issues
    print("Response structure:")
    print(f"Keys in response: {list(response.keys())}")
    if 'output' in response:
        print(f"Keys in response['output']: {list(response['output'].keys())}")
        if 'message' in response['output']:
            print(f"Keys in response['output']['message']: {list(response['output']['message'].keys())}")
            if 'toolUses' in response['output']['message']:
                print(f"Number of tool uses: {len(response['output']['message']['toolUses'])}")
            else:
                print("No 'toolUses' key found in message")
    
    return response

def process_tool_outputs(tool_responses):
    """
    Process tool outputs for display
    
    Args:
        tool_responses (list): List of tool responses
        
    Returns:
        str: Formatted display of tool responses
    """
    if not tool_responses:
        return "No tool calls made."
    
    output = "### Tool Results\n\n"
    for i, response in enumerate(tool_responses, 1):
        output += f"**Tool Call {i}: {response['tool_name']}**\n\n"
        output += f"Input: `{json.dumps(response['tool_input'])}`\n\n"
        if "error" in response["tool_output"]:
            output += f"Error: {response['tool_output']['error']}\n\n"
        else:
            output += f"Output: `{json.dumps(response['tool_output'], indent=2)}`\n\n"
    
    return output

def display_claude_tool_response(response):
    """
    Display Claude's response with detailed tool calls and results
    
    Args:
        response (dict): The API response from Claude
    """
    # Extract metrics
    elapsed_time = response.get('_elapsed_time', 0)
    input_tokens = response.get('usage', {}).get('inputTokens', 0)
    output_tokens = response.get('usage', {}).get('outputTokens', 0)
    total_tokens = response.get('usage', {}).get('totalTokens', 0)
    
    input_cost = input_tokens * 0.000003  # $3 per million tokens
    output_cost = output_tokens * 0.000015  # $5 per million tokens
    total_cost = input_cost + output_cost
    
    # Display metrics
    display(Markdown(f"### Response (in {elapsed_time:.2f} seconds)"))
    display(Markdown(f"**Tokens**: {total_tokens:,} total ({input_tokens:,} input, {output_tokens:,} output)"))
    display(Markdown(f"**Estimated cost**: ${total_cost:.5f}"))
    
    # Extract the text response
    result_text = "No response content found"
    if response.get('output', {}).get('message', {}).get('content'):
        content_blocks = response['output']['message']['content']
        for block in content_blocks:
            if 'text' in block:
                result_text = block['text']
                break
    
    # Display Claude's response
    display(Markdown("### Claude's Response:"))
    display(Markdown(result_text))
    
    # Extract tool calls if any
    tool_calls = []
    tool_responses = []
    
    # Look for tool calls in the response
    message = response.get('output', {}).get('message', {})
    
    # Check for potential alternative ways tool use might be structured in the response
    tool_uses = message.get('toolUses', [])
    if not tool_uses and 'content' in message:
        # Look for potential tool use blocks in content
        for block in message['content']:
            if 'toolUse' in block:
                display(Markdown("\n**Note**: Found tool use in content block instead of toolUses"))
                # Extract information from the content block
                tool_use_block = block['toolUse']
                tool_name = tool_use_block.get('name', 'Unknown tool')
                tool_input = tool_use_block.get('input', {})
                tool_id = tool_use_block.get('toolUseId', 'unknown-id')
                
                tool_uses.append({
                    'name': tool_name,
                    'input': tool_input,
                    'toolUseId': tool_id
                })
    
    if tool_uses:
        # Display detailed tool call information
        display(Markdown("\n### 🛠️ Tool Calls Details"))
        
        for i, tool_use in enumerate(tool_uses, 1):
            tool_name = tool_use.get('name', 'Unknown tool')
            tool_input = tool_use.get('input', {})
            tool_id = tool_use.get('toolUseId', 'unknown-id')
            
            # Format the JSON input with proper indentation for better readability
            formatted_input = json.dumps(tool_input, indent=2)
            
            # Display detailed tool information in a code block
            display(Markdown(f"#### Tool Call {i}: `{tool_name}`"))
            display(Markdown(f"**Tool ID**: `{tool_id}`"))
            display(Markdown("**JSON Input**:"))
            display(Markdown(f"```json\n{formatted_input}\n```"))
            
            # Process the tool call
            tool_output = handle_tool_call(tool_name, tool_input)
            
            # Format the tool output
            if isinstance(tool_output, dict):
                formatted_output = json.dumps(tool_output, indent=2)
                display(Markdown("**Tool Output**:"))
                display(Markdown(f"```json\n{formatted_output}\n```"))
            else:
                display(Markdown(f"**Tool Output**: {tool_output}"))
            
            # Add to our tracking lists
            tool_calls.append({
                "tool_name": tool_name,
                "tool_input": tool_input,
                "tool_id": tool_id
            })
            
            tool_responses.append({
                "tool_name": tool_name,
                "tool_input": tool_input,
                "tool_output": tool_output,
                "tool_id": tool_id
            })
    else:
        display(Markdown("\n### No Tool Calls Found in Response"))
        display(Markdown("Note: This might indicate that either:"))
        display(Markdown("1. Claude chose not to use any tools for this query"))
        display(Markdown("2. There's an issue with how tool calls are structured in the response"))
        
        # Print more details about the response structure to help diagnose issues
        if 'output' in response and 'message' in response['output']:
            message_keys = list(response['output']['message'].keys())
            display(Markdown(f"Message keys available: {message_keys}"))
    
    return {
        "text_response": result_text,
        "tool_calls": tool_calls,
        "tool_responses": tool_responses
    }

In [ ]:
# Very explicit prompt that should require tool use
explicit_tool_prompt = """
Please calculate 1234 * 5678 using the calculator tool.
Do not try to calculate this yourself - use the calculator tool.
"""

# Invoke Claude with the updated functions
calculator_response = invoke_claude_with_tools(
    explicit_tool_prompt,
    [calculator_tool],
    reasoning_budget=2048
)

# Display the response
calculator_results = display_claude_tool_response(calculator_response)

## 拡張思考による基本的なツールの使用の実装

拡張思考の後にツールを使用するという基本的なパターンを示す簡単な例から始めましょう。Claude に推論と計算の両方を必要とする問題を与えます。

In [ ]:
# Simple example: Calculate compound interest
compound_interest_prompt = """
I need to calculate how much money I'll have if I invest $10,000 for 5 years
at an annual interest rate of 7%, compounded quarterly.

First explain the compound interest formula and how to apply it in this case,
then use the calculator tool to perform the calculation.
"""

# Invoke Claude with the calculator tool
calculator_response = invoke_claude_with_tools(
    compound_interest_prompt,
    [calculator_tool],
    reasoning_budget=2048
)

# Display the response
calculator_results = display_claude_tool_response(calculator_response)

#### 別の例を追加してみましょう:

In [ ]:
# Example: Weather comparison
weather_prompt = """
I'm trying to decide whether to vacation in Paris, France or Tokyo, Japan next month.
Can you compare the current weather conditions in both cities and give me your recommendation?
Please consider temperature, weather conditions, and humidity in your analysis.
"""

# Invoke Claude with the weather tool
weather_response = invoke_claude_with_tools(
    weather_prompt,
    [weather_tool],
    reasoning_budget=2048
)

# Display the response
weather_results = display_claude_tool_response(weather_response)

## チェックイン ポイント: 基本的なツール使用パターン

前の例で Claude がツールを使用した方法について、何に気付きましたか?

1. Claude は最初に拡張思考を使用して問題を分析しました (出力には表示されませんが、舞台裏で行われています)

2. 次に、その推論とアプローチを説明するテキスト応答を提供しました

3. 最後に、必要な情報を取得するためにツール呼び出しを行いました

4. ツール呼び出しは、推論に基づいて焦点が絞られ、具体的でした

この連続パターンは、次の理由で強力です:

- 拡張思考により、Claude は必要な情報を理解するのに役立ちます
- ツール呼び出しは、より焦点が絞られ、関連性があります
- 推論により、ツールの使用に関するコンテキストと説明が提供されます

では、複数の推論とツール使用のステップを組み合わせた、より複雑なワークフローに進みましょう。

## マルチステップ ワークフローの構築

実際のアプリケーションでは、多くの場合、複数のステップの推論とツールの使用が必要です。次のものを組み合わせたマルチステップ ワークフローを実装してみましょう。

1. 問題についての最初の推論
2. 情報を収集するためのツール呼び出し
3. ツールの結果を分析するためのさらなる推論
4. その分析に基づく追加のツール呼び出し

これにより、複雑な問題に対処できる、より動的で強力なワークフローが作成されます。

In [ ]:
def run_multi_step_workflow(initial_prompt, tools, max_steps=3):
    """
    Run a multi-step workflow with reasoning and tool use
    
    Args:
        initial_prompt (str): The initial user prompt
        tools (list): List of tools to make available
        max_steps (int): Maximum number of conversation turns
        
    Returns:
        list: The conversation history
    """
    conversation = []
    current_prompt = initial_prompt
    
    for step in range(max_steps):
        print(f"\n--- Step {step+1} of {max_steps} ---\n")
        
        # Invoke Claude with the current prompt
        response = invoke_claude_with_tools(
            current_prompt,
            tools,
            reasoning_budget=4096
        )
        
        # Display the response
        results = display_claude_tool_response(response)
        conversation.append({
            "prompt": current_prompt,
            "response": response,
            "results": results
        })
        
        # Check if tool calls were made
        if not results["tool_calls"]:
            print("\nNo tool calls made. Workflow complete.")
            break
        
        # Process tool results and create the next prompt
        tool_summary = "\n\nHere are the results from the tools you used:\n\n"
        for tool_response in results["tool_responses"]:
            formatted_output = json.dumps(tool_response['tool_output'], indent=2)
            tool_summary += f"- {tool_response['tool_name']} tool returned:\n```json\n{formatted_output}\n```\n\n"
        
        current_prompt = f"""
        Thank you for your previous response. Here are the results from the tools you used:
        
        {tool_summary}
        
        Based on these results, please continue your analysis. You may use the tools again if needed.
        """
        
        # Ask if the user wants to continue to the next step
        if step < max_steps - 1:
            print("\nContinuing to next step...\n")
    
    print("\nWorkflow complete.")
    return conversation

#### 次に、これを複数ステップの調査の例に使用してみましょう。

In [ ]:
# Multi-step research example
research_prompt = """
I'm doing research on the impacts of climate change on agriculture.

Please help me gather information about:
1. Basic facts about climate change and its main causes
2. Specific impacts on agricultural productivity
3. Calculate approximately how much global crop yields might decrease if temperatures rise by 2°C

Use tools as needed to get this information.
"""

# Run the multi-step workflow
research_conversation = run_multi_step_workflow(
    research_prompt,
    [calculator_tool, wikipedia_tool],
    max_steps=3
)

## チェックイン ポイント: マルチステップ ワークフロー

マルチステップ ワークフローの例では、Claude が次の操作を実行できることを確認できました。

1. 基本的な情報を収集するために最初のツール呼び出しを行う
2. これらのツールからの結果を処理する
3. 新しい情報に基づいて追加のツール呼び出しを行う
4. 複数のターンにわたって包括的な応答を構築する

このパターンは、次のような複雑な調査および分析タスクに効果的です。
- 複数のソースから情報を収集する必要がある
- 計算またはデータ処理が必要
- 以前の調査結果に基づいて連続したステップを構築する

ただし、このアプローチには課題があります。
- 複数のターンにわたって会話のコンテキストを管理する
- 使用されたツールと収集された情報を追跡する
- 潜在的なエラーや不足している情報を処理する

次に、エラーを処理し、より堅牢なワークフローを構築するための戦略を見てみましょう。

## エラー処理と回復戦略

拡張思考とツールの使用を組み合わせると、さまざまなエラーが発生する可能性があります。

1. **ツール実行エラー**: ツールが失敗したり、予期しない結果が返されたりする可能性があります
2. **推論エラー**: クロードが問題またはツールの結果を誤って解釈する可能性があります
3. **調整エラー**: 推論とツールの使用の順序が同期しなくなる可能性があります

これらのエラーを処理し、より堅牢なワークフローを作成するための戦略をいくつか実装しましょう。

In [ ]:
def invoke_claude_with_error_handling(prompt, tools, reasoning_budget=4096, max_retries=2):
    """
    Invoke Claude with extended thinking and tool use, with error handling
    
    Args:
        prompt (str): The user prompt
        tools (list): List of tools to make available
        reasoning_budget (int): Token budget for reasoning
        max_retries (int): Maximum number of retry attempts for tool errors
        
    Returns:
        dict: Results including the response and any error information
    """
    try:
        # First attempt
        response = invoke_claude_with_tools(
            prompt,
            tools,
            reasoning_budget=reasoning_budget
        )
        
        # Display the initial response
        results = display_claude_tool_response(response)
        
        # Check for tool errors
        tool_errors = []
        for tool_response in results.get("tool_responses", []):
            if isinstance(tool_response.get("tool_output", {}), dict) and "error" in tool_response.get("tool_output", {}):
                tool_errors.append({
                    "tool_name": tool_response["tool_name"],
                    "error": tool_response["tool_output"]["error"],
                    "input": tool_response["tool_input"]
                })
        
        # If there are tool errors, attempt recovery
        retry_count = 0
        while tool_errors and retry_count < max_retries:
            retry_count += 1
            print(f"\n--- Retry attempt {retry_count} for tool errors ---\n")
            
            # Create a prompt explaining the errors
            error_prompt = f"""
            There were some issues with the tools you tried to use:
            
            {json.dumps(tool_errors, indent=2)}
            
            Please try again with different approaches or parameters. Here was the original request:
            
            {prompt}
            """
            
            # Retry with the error information
            retry_response = invoke_claude_with_tools(
                error_prompt,
                tools,
                reasoning_budget=reasoning_budget
            )
            
            # Display the retry response
            retry_results = display_claude_tool_response(retry_response)
            
            # Check if errors were resolved
            new_tool_errors = []
            for tool_response in retry_results.get("tool_responses", []):
                if isinstance(tool_response.get("tool_output", {}), dict) and "error" in tool_response.get("tool_output", {}):
                    new_tool_errors.append({
                        "tool_name": tool_response["tool_name"],
                        "error": tool_response["tool_output"]["error"],
                        "input": tool_response["tool_input"]
                    })
            
            # If errors were resolved or reduced, update the results
            if len(new_tool_errors) < len(tool_errors):
                results = retry_results
                tool_errors = new_tool_errors
            else:
                # If no improvement, stop retrying
                break
        
        # Add error summary to results
        results["tool_errors"] = tool_errors
        results["retry_count"] = retry_count
        
        return results
    
    except Exception as e:
        # Handle any unexpected errors
        print(f"Error in invoke_claude_with_error_handling: {str(e)}")
        return {
            "error": str(e),
            "text_response": f"An error occurred: {str(e)}",
            "tool_calls": [],
            "tool_responses": [],
            "tool_errors": [],
            "retry_count": 0
        }

#### 例を使ってエラー処理をテストしてみましょう。

In [ ]:
# Example with potential errors
error_prone_prompt = """
I want to calculate how much 1,000,000 raised to the power of 100 is.
Use the calculator tool to figure this out.
"""

# Invoke Claude with error handling
error_handling_results = invoke_claude_with_error_handling(
    error_prone_prompt,
    [calculator_tool],
    max_retries=2
)

# Display error summary
if error_handling_results.get("tool_errors"):
    print("\n### Error Summary")
    print(f"Total errors: {len(error_handling_results['tool_errors'])}")
    print(f"Retry attempts: {error_handling_results['retry_count']}")
    for i, error in enumerate(error_handling_results["tool_errors"], 1):
        print(f"\nError {i}:")
        print(f"Tool: {error['tool_name']}")
        print(f"Input: {error['input']}")
        print(f"Error: {error['error']}")
else:
    print("\nNo tool errors occurred or all errors were resolved.")

## 基本的なリサーチ エージェントの構築

では、これまでに学んだすべての概念を組み合わせて、次のようなシンプルなリサーチ エージェントを構築してみましょう。

1. 拡張思考を使用してリサーチ クエリを理解する
2. 適切なツールを呼び出して情報を収集する
3. 追加の推論を使用して結果を処理する
4. エラーを適切に処理する
5. 包括的な応答を生成する

このエージェントは、拡張思考とツールの使用を組み合わせることで強力なアプリケーションを作成できることを示しています。

In [ ]:
class ResearchAgent:
    """
    A simple research agent that combines extended thinking with tool use
    """
    def __init__(self, tools=None, reasoning_budget=4096):
        """
        Initialize the research agent
        
        Args:
            tools (list): List of tools available to the agent
            reasoning_budget (int): Token budget for reasoning
        """
        self.tools = tools or [calculator_tool, weather_tool, wikipedia_tool]
        self.reasoning_budget = reasoning_budget
        self.conversation_history = []
    
    def research(self, query, max_steps=3):
        """
        Conduct research on a query
        
        Args:
            query (str): The research query
            max_steps (int): Maximum number of conversation turns
            
        Returns:
            dict: The research results
        """
        print(f"Starting research on: {query}")
        
        # Format the initial prompt
        initial_prompt = f"""
        You are a research assistant. Please help me research the following topic:
        
        {query}
        
        Think carefully about what information you need to gather and which tools would be most helpful.
        Develop a research plan, then execute it step by step using the available tools.
        
        Provide a comprehensive response that synthesizes the information you gather.
        """
        
        # Run the multi-step workflow
        conversation = run_multi_step_workflow(
            initial_prompt,
            self.tools,
            max_steps=max_steps
        )
        
        # Store the conversation history
        self.conversation_history.extend(conversation)
        
        # Extract the final results
        if conversation:
            final_step = conversation[-1]
            final_response = final_step.get("results", {}).get("text_response", "No response generated")
            
            # Create a summary of all tool calls
            tool_calls_summary = []
            for step in conversation:
                for tool_call in step.get("results", {}).get("tool_calls", []):
                    tool_calls_summary.append({
                        "tool": tool_call["tool_name"],
                        "input": tool_call["tool_input"]
                    })
            
            return {
                "query": query,
                "steps": len(conversation),
                "final_response": final_response,
                "tool_calls": tool_calls_summary,
                "full_conversation": conversation
            }
        else:
            return {
                "query": query,
                "steps": 0,
                "final_response": "Research could not be completed",
                "tool_calls": [],
                "full_conversation": []
            }
    
    def summarize_research(self, results):
        """
        Generate a summary of the research results
        
        Args:
            results (dict): The research results
            
        Returns:
            str: A markdown summary of the research
        """
        if not results:
            return "No research results available."
        
        summary = f"""
        # Research Summary: {results['query']}
        
        ## Overview
        - Research steps: {results['steps']}
        - Tools used: {len(results['tool_calls'])}
        
        ## Tools Used
        """
        
        # Group tool calls by tool type
        tool_usage = {}
        for call in results['tool_calls']:
            tool_name = call['tool']
            if tool_name not in tool_usage:
                tool_usage[tool_name] = []
            tool_usage[tool_name].append(call['input'])
        
        # Add tool usage to summary
        for tool_name, calls in tool_usage.items():
            summary += f"\n### {tool_name.capitalize()}\n"
            summary += f"- Used {len(calls)} times\n"
            for i, call in enumerate(calls[:3], 1):  # Show up to 3 examples
                summary += f"- Example {i}: `{json.dumps(call)}`\n"
            if len(calls) > 3:
                summary += f"- ...and {len(calls) - 3} more\n"
        
        # Add the final response
        summary += f"\n## Final Response\n\n{results['final_response']}\n"
        
        return summary

#### では、リサーチクエリを使って ResearchAgent をテストしてみましょう。

In [ ]:
# Create a research agent with our tools
agent = ResearchAgent(
    tools=[calculator_tool, wikipedia_tool],
    reasoning_budget=4096
)

# Run research on a topic
research_results = agent.research(
    "What are the potential economic impacts of artificial intelligence on employment in the next decade?",
    max_steps=2
)

# Display the research summary
research_summary = agent.summarize_research(research_results)
display(Markdown(research_summary))

## チェックイン ポイント: 効果的なリサーチ エージェントの構築

当社のリサーチ エージェントは、拡張思考とツールの使用を組み合わせるためのいくつかの重要な原則を示しています:

1. **リサーチ プランニング**: 拡張思考を使用して体系的なリサーチ プランを作成します
2. **ツールの選択**: 必要な情報に基づいて適切なツールを選択します
3. **マルチステップ ワークフロー**: 複数の会話ターンにわたって知識を構築します
4. **情報の統合**: さまざまなソースからの情報を組み合わせて、一貫した応答を作成します

より高度なリサーチ エージェントを構築するには、このアプローチを次の方法で拡張できます:

- より専門的なツール (例: データ分析、視覚化)
- 会話ターン全体にわたるコンテキスト管理の改善
- より洗練されたエラー処理および回復戦略
- 過去のリサーチから学習して将来のクエリを改善します

拡張思考の後にツールを使用するというコア パターンは、これらのより高度な機能の強力な基盤となります。

## 結論: 拡張思考とツールの使用を組み合わせる力

このノートブックでは、Claude 3.7 Sonnet の拡張思考能力とツールの使用を効果的に組み合わせる方法を検討しました。主なポイントは次のとおりです:

1. **シーケンシャル パターン**: Claude はまず問題について推論し、次にツールを使用して情報を収集したりアクションを実行したりします
2. **マルチステップ ワークフロー**: 複雑な問題は、推論とツールの使用を複数ステップ行うことで解決できます
3. **エラー処理**: 堅牢な調査エージェントには、エラーを処理して問題から回復するための戦略が必要です
4. **調査計画**: 拡張思考により、Claude はツールの使用をガイドする体系的な調査計画を作成できます

この組み合わせにより、人間が複雑なタスクに取り組む方法を模倣した強力なワークフローが作成されます:
- 最初に必要な情報を検討します
- 次に、適切なツールを使用してその情報を収集します
- 最後に、情報を総合的に統合して包括的な応答を作成します

Claude 3.7 Sonnet を使用して独自のアプリケーションを構築するときは、このパターンを活用して、よりインテリジェントで適応性に優れ、便利な AI アシスタントを作成する方法を検討してください。

## 次のステップ

拡張思考とツールの使用をさらに探求するには、次の点を検討してください。

1. 特定のドメインに合わせて調整された **より専門的なツールの追加**
2. **より洗練されたエラー処理** 戦略の実装
3. さまざまなタイプのタスクに対して **異なる推論予算の実験**
4. 特定のユースケース向けに **より複雑なマルチステップ ワークフローの構築**

拡張思考とツールの使用を組み合わせることで、複雑な現実の問題に対処できる AI アシスタントを作成するための幅広い可能性が開かれます。